Interrogate the impact of the COVID lockdown in the economy.

In [1]:
from datetime import date
import pandas as pd

from src.data.download_stock import download_stocks
from src.data.preprocessing import cacl_monthly_avg_kpi
from src.models.causal_impact import causal_impact_analysis
from src.utils.plots import timeline_px
from src.utils.variables import pharma_companies, consumer_staples, retail, global_economy_indicators

In [2]:
industry_tickers = {"Pharma" : pharma_companies,
                    "Consumables" : consumer_staples,
                    "Retail" : retail,
                    "Global Economy" : global_economy_indicators}

tickers = (list(pharma_companies.keys())
          + list(consumer_staples.keys())
          + list(retail.keys())
          + list(global_economy_indicators.keys()))

In [3]:
raw_dir = "./data/raw/"
start_date = date(2017, 1, 1)
end_date = date(2020, 12, 30)
event_date = date(2020, 3, 1) # Covid lockdown

Download stock market data from yahoo finance

In [4]:
# Download stock market data
download_stocks(start_date, end_date, tickers, raw_dir)

[*********************100%%**********************]  24 of 24 completed


Processing stock: 100%|██████████| 24/24 [00:00<00:00, 195.83it/s]


In [5]:
kpi= "Adj Close"

monthly_abj_close = pd.DataFrame()
for ticker in tickers:
    ticker_df = pd.read_csv(f"{raw_dir}{ticker}.csv", index_col=0)
    ticker_df.index = pd.to_datetime(ticker_df.index)
    ticker_df  = cacl_monthly_avg_kpi(ticker_df, kpi)
    ticker_df.name = ticker
    
    monthly_abj_close = pd.concat([monthly_abj_close, ticker_df], axis=1)

Plot timeline highlighting the start date of the first Covid lockdown, March 2020

In [6]:
event_str = event_date.strftime(format="%Y-%m")
labels = {"variable":"", "value":kpi, "index":"Month"}

for name, tickers_dict in industry_tickers.items():
    tickers = list(tickers_dict.keys())
    df = monthly_abj_close[tickers]
    write_html =  f"./figures/{name.lower()}.html"
    timeline_px(df, event_str, name, write_html, labels)

In [7]:
def covid_causal_impact(tickers_dict:dict):
    pre_period = [start_date, date(event_date.year, event_date.month-1, 1)]
    pre_period = [pd.to_datetime(x) for x in pre_period]
    post_period = [event_date, date(end_date.year, end_date.month-1, 1)]
    post_period = [pd.to_datetime(x) for x in post_period]

    for company in tickers_dict.keys():
        print(f"===== {tickers_dict[company]} =====")
        tickers = [company,]
        tickers += list(global_economy_indicators.keys())
        
        df = monthly_abj_close[tickers]
        df.dropna(axis=1, inplace=True)
        ci = causal_impact_analysis(df, company, pre_period, post_period)
        
        ci.summary(output="summary", path=f"./figures/python/{company}.summary.txt")
        ci.summary(output="report", path=f"./figures/python/{company}.report.txt") 
            
        ci.plot(panels=['original', 'pointwise', 'cumulative'], figsize=(15,5), 
                fname=f"./figures/python/{company}.png")    

In [8]:
covid_causal_impact(pharma_companies)

===== Pfizer Inc. =====


C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



The mean absolute percentage error in the pre period is 0.03%
                               Average      Cumulative
Actual                              30             270
Predicted                           23             208
95% CI                        [18, 27]      [168, 248]
                                                      
Absolute Effect                      6              62
95% CI                         [11, 2]       [102, 22]
                                                      
Relative Effect                  30.0%           30.0%
95% CI                  [49.1%, 10.8%]  [49.1%, 10.8%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                              30             270
Predicted                           23             208
95% CI                        [18, 27]      [168, 248

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Novartis AG =====
The mean absolute percentage error in the pre period is 0.02%
                               Average      Cumulative
Actual                              75             680
Predicted                           71             642
95% CI                        [64, 78]      [576, 708]
                                                      
Absolute Effect                      4              37
95% CI                        [11, -3]      [103, -28]
                                                      
Relative Effect                   5.9%            5.9%
95% CI                  [16.2%, -4.4%]  [16.2%, -4.4%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                              75             680
Predicted                           71             642
95% CI                       

...

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Eli Lilly and Company =====
The mean absolute percentage error in the pre period is 0.04%
                               Average      Cumulative
Actual                             142            1282
Predicted                          130            1173
95% CI                      [112, 147]    [1016, 1330]
                                                      
Absolute Effect                     12             109
95% CI                        [29, -5]      [266, -47]
                                                      
Relative Effect                   9.3%            9.3%
95% CI                  [22.7%, -4.1%]  [22.7%, -4.1%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                             142            1282
Predicted                          130            1173
95% CI             

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== AbbVie Inc. =====
The mean absolute percentage error in the pre period is 0.05%
                               Average      Cumulative
Actual                              77             700
Predicted                           89             808
95% CI                       [72, 107]      [651, 965]
                                                      
Absolute Effect                    -12            -108
95% CI                        [5, -29]      [48, -265]
                                                      
Relative Effect                 -13.4%          -13.4%
95% CI                  [6.0%, -32.9%]  [6.0%, -32.9%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                              77             700
Predicted                           89             808
95% CI                       

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



In [9]:
covid_causal_impact(retail)

===== Amazon.com, Inc. =====
The mean absolute percentage error in the pre period is 0.03%
                               Average      Cumulative
Actual                             138            1247
Predicted                          135            1216
95% CI                      [124, 146]    [1118, 1314]
                                                      
Absolute Effect                      3              30
95% CI                        [14, -7]      [128, -66]
                                                      
Relative Effect                   2.5%            2.5%
95% CI                  [10.6%, -5.5%]  [10.6%, -5.5%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                             138            1247
Predicted                          135            1216
95% CI                  

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Industria de Diseño Textil, S.A. =====
The mean absolute percentage error in the pre period is 0.03%
                               Average      Cumulative
Actual                              21             196
Predicted                           25             229
95% CI                        [21, 29]      [195, 263]
                                                      
Absolute Effect                     -3             -33
95% CI                         [0, -7]        [0, -67]
                                                      
Relative Effect                 -14.5%          -14.5%
95% CI                  [0.3%, -29.3%]  [0.3%, -29.3%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                              21             196
Predicted                           25             229
95% CI  

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Hennes & Mauritz AB =====
The mean absolute percentage error in the pre period is 0.05%
                                Average       Cumulative
Actual                                2               24
Predicted                             2               25
95% CI                           [1, 3]         [17, 32]
                                                        
Absolute Effect                       0                0
95% CI                           [0, 0]          [6, -8]
                                                        
Relative Effect                   -3.7%            -3.7%
95% CI                  [26.0%, -33.4%]  [26.0%, -33.4%]
                                                        
P-value                            0.0%                 
Prob. of Causal Effect           100.0%                 
None
                                Average       Cumulative
Actual                                2               24
Predicted                             2       

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== LVMH Moët Hennessy Louis Vuitton S.E. =====
The mean absolute percentage error in the pre period is 0.03%
                              Average     Cumulative
Actual                            368           3316
Predicted                         327           2946
95% CI                     [290, 363]   [2617, 3274]
                                                    
Absolute Effect                    41            370
95% CI                        [77, 4]      [698, 42]
                                                    
Relative Effect                 12.6%          12.6%
95% CI                  [23.7%, 1.4%]  [23.7%, 1.4%]
                                                    
P-value                          0.0%               
Prob. of Causal Effect         100.0%               
None
                              Average     Cumulative
Actual                            368           3316
Predicted                         327           2946
95% CI                     [290, 36

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Fast Retailing Co., Ltd. =====
The mean absolute percentage error in the pre period is 0.06%
                               Average       Cumulative
Actual                           19835           178516
Predicted                        12310           110794
95% CI                   [8491, 16129]  [76423, 145165]
                                                       
Absolute Effect                   7524            67721
95% CI                   [11343, 3705]  [102092, 33350]
                                                       
Relative Effect                  61.1%            61.1%
95% CI                  [92.1%, 30.1%]   [92.1%, 30.1%]
                                                       
P-value                           0.0%                 
Prob. of Causal Effect          100.0%                 
None
                               Average       Cumulative
Actual                           19835           178516
Predicted                        12310           110794


C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



In [10]:
covid_causal_impact(consumer_staples)

===== The Coca-Cola Company (KO) =====
The mean absolute percentage error in the pre period is 0.03%
                                Average       Cumulative
Actual                               43              387
Predicted                            49              441
95% CI                         [44, 53]       [401, 481]
                                                        
Absolute Effect                      -5              -53
95% CI                        [-1, -10]       [-13, -93]
                                                        
Relative Effect                  -12.2%           -12.2%
95% CI                  [-3.1%, -21.3%]  [-3.1%, -21.3%]
                                                        
P-value                            0.0%                 
Prob. of Causal Effect           100.0%                 
None
                                Average       Cumulative
Actual                               43              387
Predicted                            49

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Colgate-Palmolive Company (CL): =====
The mean absolute percentage error in the pre period is 0.02%
                               Average      Cumulative
Actual                              68             620
Predicted                           61             551
95% CI                        [53, 68]      [483, 620]
                                                      
Absolute Effect                      7              68
95% CI                         [15, 0]        [137, 0]
                                                      
Relative Effect                  12.4%           12.4%
95% CI                  [24.9%, -0.0%]  [24.9%, -0.0%]
                                                      
P-value                           0.0%                
Prob. of Causal Effect          100.0%                
None
                               Average      Cumulative
Actual                              68             620
Predicted                           61             551
95% CI   

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Kimberly-Clark Corporation (KMB) =====
The mean absolute percentage error in the pre period is 0.02%
                              Average     Cumulative
Actual                            126           1134
Predicted                         111           1002
95% CI                      [98, 123]    [890, 1114]
                                                    
Absolute Effect                    14            131
95% CI                        [27, 2]      [243, 19]
                                                    
Relative Effect                 13.1%          13.1%
95% CI                  [24.3%, 2.0%]  [24.3%, 2.0%]
                                                    
P-value                          0.0%               
Prob. of Causal Effect         100.0%               
None
                              Average     Cumulative
Actual                            126           1134
Predicted                         111           1002
95% CI                      [98, 123]   

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.



===== Unilever (UL) =====
The mean absolute percentage error in the pre period is 0.03%
                              Average     Cumulative
Actual                             50            451
Predicted                          44            397
95% CI                       [39, 49]     [351, 442]
                                                    
Absolute Effect                     5             53
95% CI                        [11, 0]        [99, 8]
                                                    
Relative Effect                 13.5%          13.5%
95% CI                  [25.0%, 2.1%]  [25.0%, 2.1%]
                                                    
P-value                          0.0%               
Prob. of Causal Effect         100.0%               
None
                              Average     Cumulative
Actual                             50            451
Predicted                          44            397
95% CI                       [39, 49]     [351, 442]
      

C:\Users\joanp\AppData\Local\Temp\ipykernel_37992\4049619924.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\joanp\anaconda3\envs\causal_inference_ts\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning:

Unknown keyword arguments: dict_keys(['alpha']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.

